# BlueSky Flyer Basics

We want to understand how to build a *Flyer* in "BlueSky" to support various types of fly scans and remote data loggers.  The data about Flyers is spread about the standard documentation.  We need some clarity and a few examples that build complexity incrementally.

The basic notion of a Flyer is that it directs an *external controller* (we'll call the the *controller* here) to perform some data colelction process.  Usually, a *controller* is used to collect data at rates beyond the capabilities of BlueSky plans and the RunEngine.  Examples could be waveforms from a storage oscilloscope or a continuous motion scan of a diffractometer.


## Python imports and definitions <a class="anchor" id="imports" />

Here are the full set of packages to imported.  The first block are Python standard packages, then come the ophyd, BluSky, and databroker packages.  Just the parts we plan on using here.  Since this is also a tutorial, we will not rename imports or use other such shortcuts in the documentation (the online code has some shortcuts).

* Create a logger instance in case we want to investigate internal details as our code runs.
* Create an instance of the BlueSky RunEngine.
* Create an instance of the databroker using the `mongodb_config.yml` file on the local machine
* Arrange for the databroker to receive all events from the RunEngine


In [1]:
import logging
import threading
import time

import ophyd
import bluesky
import bluesky.plans
import databroker

logger = logging.getLogger()
RE = bluesky.RunEngine({})
db = databroker.Broker.named("mongodb_config")
RE.subscribe(db.insert)

0

### Bare Minimum Requirements for a Flyer <a class="anchor" id="flyer-requirements" />

In BlueSky, a [Flyer](http://nsls-ii.github.io/bluesky/async.html?highlight=flyer#flying) is an `ophyd.Device` that meets the Flyer interface, which has three methods:

1. Kickoff - begin accumulating data
1. Complete - BlueSky tells the Flyer that BlueSky is ready to receive data
1. Collect - the device provides the data to BlueSky

The first two methods [must return](http://nsls-ii.github.io/bluesky/hardware.html?highlight=flyer#kickoff) an instance of `ophyd.DeviceStatus` (a.k.a. a *status* object).  

The `collect()` method requires a companion `describe_collect()` that informs the RunEngine what kind of data to expect from `collect()`.

This example (which does absolutely nothing) meets the bare minimum requirement.

In [2]:
class BareMinimumFlyer(ophyd.Device):

    def kickoff(self):
        kickoff_status = ophyd.DeviceStatus(self)
        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        complete_status = ophyd.DeviceStatus(self)
        complete_status._finished(success=True)
        return complete_status

    def collect(self):
        yield {'data':{}, 'timestamps':{}, 'time':time.time()}
    
    def describe_collect(self):
        return {self.name: {}}


flyer = BareMinimumFlyer(name="flyer")
print(flyer.complete())
print(list(flyer.collect()))

# if this next step succeeds, it's proof that we did this right!
RE(bluesky.plans.fly([flyer]))

DeviceStatus(device=flyer, done=True, success=True)
[{'data': {}, 'timestamps': {}, 'time': 1524799675.8524897}]


('6e834c3d-7d2e-47e8-9902-7e726b706ae6',)

### Flyer : a starting template <a class="anchor" id="flyer-template" />

The `BareMinimumFlyer` is a good start to use a Flyer but we'll need to add a few more things to make a good template.  The first thing to do is to make the status object known to any method of the class.  We'll call it `self._completion_status` and it will tell us if the *controller* is finished.  In the constructor (`__init__()`), we set it to `None`, the value we expect when not *flying*.  Since we **need** a constructor, we must remember to call the constructor of the superclass as well or our `ophyd.Device` will not work correctly.

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self._completion_status = None

Our *controller* signals through EPICS that it is finished.  This could take some time (seconds to minutes, at least).  We need a way to detect this completion.  We can do that either by polling the PV or by subscribing to a callback on the completion event.  Here, we do it in a polling loop.  Since the polling loop is an activity that does not return until the busy record is done, we must do that waiting in a thread separate from that of the RunEngine.  (We do not want to block the RunEngine thread so it is free to respond to other activities, such as data from other streams or the user inerface.)  So, we run `my_activity()` in a separate thread that is called from `kickoff()`:

        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

The basic outline of `my_activity()` is:

    def my_activity(self):
        # set the busy record to busy (very fast)
        # wait for busy record to be done (could be very slow)
        self._completion_status._finished(success=True)

The waiting step will *block the thread* in which `my_activity()` is running but that's OK since this is separate from the RunEngine's thread.

We've also added some diagnostic reporting (calls to `logger.info(...)`) to build out the next example:

In [3]:
class MyFlyer(ophyd.Device):
    """
    starting template for a Flyer that we understand
    """

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self._completion_status = None

    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the BlueSky RunEngine.
        """
        logger.info("activity()")
        if self._completion_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # TODO: do the activity here
        # TODO: wait for completion
        
        self._completion_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self._completion_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        self._completion_status = ophyd.DeviceStatus(self)
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        kickoff_status = ophyd.DeviceStatus(self)
        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self._completion_status is None:
            raise RuntimeError("No collection in progress")

        return self._completion_status

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        self._completion_status = None
        yield {'data':{}, 'timestamps':{}, 'time':time.time()}
    
    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        return {self.name: {}}


In [4]:
ifly = MyFlyer(name="ifly")

### Diagnostics  <a class="anchor" id="Diagnostics" />

When building a `Flyer`, it is useful to have some diagnostics in place.  Already, we have been using some of these, including printing interim messages via calls to `logger.info(...)`.  Another useful diagnostic step is to call each of the methods individually to make sure they are acting as expected.

1. create an instance of the `Flyer`

    flyer = MyFlyer(name="flyer")

1. verify that `kickoff()` returns a status that is "Done"

    status = flyer.kickoff()
    status.done

1. verify that `complete()` returns a status that is "Done"

    status = flyer.complete()
    status.done

1. verify that `describe_collect()` returns a dictionary

    d = flyer.describe_collect()
    d

1. verify that `collect()` returns a generator

    g = flyer.collect()
    g

1. verify that generator is a list of data dictionaries

    list(g)


Apply some of those steps here (we'll skip testing the `ifly.complete()` method when not flying since it raises a `RuntimeError` exception if data collection is not in progress):

In [5]:
ifly.describe_collect()

{'ifly': {}}

In [6]:
list(ifly.collect())

[{'data': {}, 'time': 1524799676.1651113, 'timestamps': {}}]

Now, run this fly scan:

In [7]:
RE(bluesky.plans.fly([ifly]))

('ca09ed67-c05f-4ea0-a185-fdc247c37b7a',)

In [8]:
db[-1].stream_names

['ifly']

In [9]:
db[-1].table("ifly")

""
seq_num


## First working Flyer - trivial data <a class="anchor" id="trivial-data-flyer" />

See GitHub for a [summary of changes in source code](https://github.com/prjemian/ipython_mintvm/compare/062d1765023a4d9...388eb30304e51).

To collect data, we need to modify both the `collect()` *and* the `describe_collect()` methods.  BlueSky needs to know what kind of data to expect from this Flyer, so that it can generate the correct `descriptor` document.

For the *most* trivial case, we'll return a single number (`1.2345`) as the result of the first working Flyer.

In the `describe_collect()` method, we create a dictionary that describes the data to be collected:

        d = dict(
            source = "fictional",
            dtype = "number",
            shape = []
        )
        return {
            'ifly': {
                "x": d
            }
        }

Then, in the `collect()` method, add the actual data collection code:

        t = time.time()
        d = dict(
            time=t,
            data=dict(x=1.2345),
            timestamps=dict(x=t)
        )
        yield d


In [10]:
class MyFlyer(ophyd.Device):
    """
    build a Flyer that we understand
    """

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self._completion_status = None

    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the BlueSky RunEngine.
        """
        logger.info("activity()")
        if self._completion_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # TODO: do the activity here
        # TODO: wait for completion
        
        self._completion_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self._completion_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        self._completion_status = ophyd.DeviceStatus(self)
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        kickoff_status = ophyd.DeviceStatus(self)
        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self._completion_status is None:
            raise RuntimeError("No collection in progress")

        return self._completion_status

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        d = dict(
            source = "fictional",
            dtype = "number",
            shape = []
        )
        return {
            'ifly': {
                "x": d
            }
        }

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        self._completion_status = None
        t = time.time()
        d = dict(
            time=t,
            data=dict(x=1.2345),
            timestamps=dict(x=t)
        )
        yield d

As before, create a new instance of the *revised* `MyFlyer` class.

In [11]:
ifly = MyFlyer(name="ifly")

In [12]:
print('output from describe_collect() : ', ifly.describe_collect())
print("list output from collect() : ", list(ifly.collect()))

output from describe_collect() :  {'ifly': {'x': {'source': 'fictional', 'dtype': 'number', 'shape': []}}}
list output from collect() :  [{'time': 1524799676.5974529, 'data': {'x': 1.2345}, 'timestamps': {'x': 1524799676.5974529}}]


Running this flyer with the RunEngine seems anticlimactic but the lack of exceptions tells us that it ran and we get a UUID at the end.

In [13]:
RE(bluesky.plans.fly([ifly]))

('80ce6cdb-5d11-4438-9c12-bf287d95a3f8',)

We next query the last scan in the databroker and show a table of the stream from `collect()`:

In [14]:
h = db[-1]
h.table(h.stream_names[0])

,time,x
seq_num,,
1,2018-04-26 22:27:56.638736,1.2345


## Flyer that "collects" 1-D array data  <a class="anchor" id="simple-1d-array-flyer" />

We continue to expand the capabilities of our working example of a Flyer.  Now, we wish to "collect" a 1-D array of data.  We'll manufacture that data ourselves to keep the code simple.

Here, we generate an array of 5 numbers, each representing the elapsed time (in seconds) since the scan started (in `kickoff()`).  We record the start time with `self.t0 = time.time()`.  Also, `time.time()` is used to generate the timestamps in the data events.  In a real case, we *might* get timestamps from the *controller* or we'd have to make it up ourselves, just like this example.

See GitHub for a [summary of changes in source code](https://github.com/prjemian/ipython_mintvm/compare/388eb30304e51...a0af3ec57a3430e777b3).

In [15]:
class MyFlyer(ophyd.Device):
    """
    a Flyer that we understand that reports 1-D array of data
    """

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self._completion_status = None
        self.t0 = 0

    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the BlueSky RunEngine.
        """
        logger.info("activity()")
        if self._completion_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # TODO: do the activity here
        # TODO: wait for completion
        
        self._completion_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self._completion_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        self._completion_status = ophyd.DeviceStatus(self)
        self.t0 = time.time()
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        kickoff_status = ophyd.DeviceStatus(self)
        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self._completion_status is None:
            raise RuntimeError("No collection in progress")

        return self._completion_status

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        d = dict(
            source = "elapsed time, s",
            dtype = "number",
            shape = (1,)
        )
        return {
            self.name: {
                "x": d
            }
        }

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        self._completion_status = None
        for _ in range(5):
            t = time.time()
            x = t - self.t0 # data is elapsed time since kickoff()
            d = dict(
                time=t,
                data=dict(x=x),
                timestamps=dict(x=t)
            )
            yield d


In [16]:
ifly = MyFlyer(name="ifly")
print('output from describe_collect() : ', ifly.describe_collect())
print("list output from collect() : ", list(ifly.collect()))

output from describe_collect() :  {'ifly': {'x': {'source': 'elapsed time, s', 'dtype': 'number', 'shape': (1,)}}}
list output from collect() :  [{'time': 1524799676.9440706, 'data': {'x': 1524799676.9440706}, 'timestamps': {'x': 1524799676.9440706}}, {'time': 1524799676.9440737, 'data': {'x': 1524799676.9440737}, 'timestamps': {'x': 1524799676.9440737}}, {'time': 1524799676.9440756, 'data': {'x': 1524799676.9440756}, 'timestamps': {'x': 1524799676.9440756}}, {'time': 1524799676.9440775, 'data': {'x': 1524799676.9440775}, 'timestamps': {'x': 1524799676.9440775}}, {'time': 1524799676.9440796, 'data': {'x': 1524799676.9440796}, 'timestamps': {'x': 1524799676.9440796}}]


Again, not much information from running this flyer, except that it succeeds and a uuid is returned.

In [17]:
RE(bluesky.plans.fly([ifly]))

('dc5c859c-036e-478b-ae50-6ee13b4c9865',)

More information is obtained by asking the databroker about the most recent scan.  Here's the data table from our stream of data events.  (The stream is named "ifly" as set from `self.name` in the `describe_collect()` method.)

In [18]:
h = db[-1]
h.table(h.stream_names[0])

,time,x
seq_num,,
1,2018-04-26 22:27:56.985228,0.011414
2,2018-04-26 22:27:56.985304,0.011490
3,2018-04-26 22:27:56.985323,0.011508
4,2018-04-26 22:27:56.985339,0.011524
5,2018-04-26 22:27:56.985355,0.011540
